### **Ice Marketing Campaign Strategy 2017**

#### **1. Introduction**
- The objective of this product is to develop an advertising strategy for projected winners in the video game sector.
- Historical video game sales data will be analyzed to inform the forecast and identify potential big winners that should garner advertising support proportional to their sales.
- The dataset is the aggregation of several open sources providing insights on user and expert reviews, platforms, and historical sales data.

#### **2. Approach**
The dataset will be analyzed to understand the following:
- Sales Performance Across Different Platforms at Different Points in Time
- Impact of User and Professional Reviews on Sales
- Profitability Across Different Genres
- Regional User Profiles

The project will consist of the following stages:
1. Data Preparation
2. Data Analysis
3. Regional User Profile Definition
4. Hypothesis Testing
5. Conclusion

#### **3. Initialization**
Importing all relevant libraries and loading in the dataset.

In [10]:
import pandas as pd
import numpy as np
import math as mt
import random as round
from matplotlib import pyplot as pyplot
from scipy import stats as st

In [11]:
#Load the data

url = "https://raw.githubusercontent.com/Flamingo-Rocker/Video-Game-Marketing-Strategy/refs/heads/main/games.csv"
games = pd.read_csv(url)

#Display first few rows the check that the pull worked
games.head()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


#### **4. Data Preparation**
- Replace column names
- Convert data to required types
- Address missing and/or duplicate values as-necessary
- Enrich the data with additional columns useful for analysis

In [12]:
# Print the general/summary information about the dataset
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB
